# Dairy Cow Milk Yield Prediction - Complete ML Pipeline

**Course**: CS 363M Machine Learning  
**Project**: Fall 2025 Kaggle Competition  
**Task**: Predict milk yield for 250,000 dairy cows  

---

## Project Overview

This notebook contains a complete machine learning workflow:
1. **Data Loading & Exploration** - Understand data structure and distribution
2. **Data Cleaning** - Handle missing values and outliers
3. **Feature Engineering** - Create and transform features
4. **Feature Selection** - Select most relevant features
5. **Model Training** - Train multiple models
6. **Model Evaluation** - Compare model performance
7. **Ensemble Learning** - Use ensemble methods to improve performance
8. **Prediction & Submission** - Generate Kaggle submission file

**Grading Criteria**:
- 10% Kaggle leaderboard score
- 90% Jupyter Notebook methodology
  - Data Cleaning (20%)
  - Data Exploration (20%)
  - Feature Engineering (20%)
  - Modeling Approach (20%)
  - Code Quality (20%)

## 0. Environment Setup and Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import (RandomForestRegressor, GradientBoostingRegressor, 
                               AdaBoostRegressor, VotingRegressor, StackingRegressor)
from scipy import stats
from scipy.stats import norm, skew
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

DATA_DIR = Path('../data')
RAW_DATA_DIR = DATA_DIR / 'raw'
PROCESSED_DATA_DIR = DATA_DIR / 'processed'
MODEL_DIR = Path('../models')
SUBMISSION_DIR = Path('../submissions')

for dir_path in [PROCESSED_DATA_DIR, MODEL_DIR, SUBMISSION_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Random state: {RANDOM_STATE}")

## 1. Initial Data Exploration

**Objective**: Load and understand the dataset structure

In this section, we will:
1. Load training and testing datasets
2. View basic information about the data
3. Check data types and missing values
4. Perform preliminary statistical analysis

In [ ]:
train = pd.read_csv(RAW_DATA_DIR / 'cattle_data_train.csv')
test = pd.read_csv(RAW_DATA_DIR / 'cattle_data_test.csv')

target_col = 'Milk_Yield_L'
id_col = 'Cattle_ID'

print("Data Loading Complete")
print(f"Training set shape: {train.shape}")
print(f"Test set shape: {test.shape}")
print(f"Target variable: {target_col}")
print(f"ID column: {id_col}")
print(f"\nColumn Validation:")
print(f"  Target variable exists: {target_col in train.columns}")
print(f"  ID column exists: {id_col in train.columns}")

In [ ]:
print("Training Set Information")
print(train.info())

print("First 5 Rows")
print(train.head())

print("Descriptive Statistics")
print(train.describe())

In [ ]:
numeric_features = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

if target_col in numeric_features:
    numeric_features.remove(target_col)
if id_col in numeric_features:
    numeric_features.remove(id_col)
if id_col in categorical_features:
    categorical_features.remove(id_col)

print("=" * 60)
print("Feature Type Identification")
print("=" * 60)
print(f"Number of numeric features: {len(numeric_features)}")
print(f"Number of categorical features: {len(categorical_features)}")
print(f"\nNumeric features:\n{numeric_features}")
print(f"\nCategorical features:\n{categorical_features}")

## Missing Value Analysis

Missing values can impact model training. We need to:
1. Calculate the number and percentage of missing values for each column
2. Visualize the distribution of missing values
3. Decide how to handle missing values

In [ ]:
missing_train = pd.DataFrame({
    'column': train.columns,
    'missing_count': train.isnull().sum(),
    'missing_percent': (train.isnull().sum() / len(train) * 100).round(2)
})
missing_train = missing_train[missing_train['missing_count'] > 0].sort_values('missing_percent', ascending=False)

if len(missing_train) > 0:
    print(missing_train.to_string(index=False))
    plt.figure(figsize=(12, 6))
    plt.barh(missing_train['column'], missing_train['missing_percent'])
    plt.xlabel('Missing Percentage (%)')
    plt.title('Training Set Missing Values Distribution')
    plt.tight_layout()
    plt.show()
else:
    print("No missing values in training set.")

missing_test = pd.DataFrame({
    'column': test.columns,
    'missing_count': test.isnull().sum(),
    'missing_percent': (test.isnull().sum() / len(test) * 100).round(2)
})
missing_test = missing_test[missing_test['missing_count'] > 0].sort_values('missing_percent', ascending=False)

print("Test set missing values summary")
if len(missing_test) > 0:
    print(missing_test.to_string(index=False))
else:
    print("No missing values in test set.")


## Target Variable Analysis

Analyze distribution characteristics of the target variable (Milk Yield):
- Distribution shape (normality test)
- Skewness and kurtosis
- Whether a transformation is needed

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].hist(train[target_col], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Milk Yield (L)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Milk Yield Distribution - Histogram')
axes[0, 0].axvline(train[target_col].mean(), color='red', linestyle='--', label=f'Mean: {train[target_col].mean():.2f}')
axes[0, 0].axvline(train[target_col].median(), color='green', linestyle='--', label=f'Median: {train[target_col].median():.2f}')
axes[0, 0].legend()

axes[0, 1].boxplot(train[target_col], vert=True)
axes[0, 1].set_ylabel('Milk Yield (L)')
axes[0, 1].set_title('Milk Yield Distribution - Boxplot')
axes[0, 1].grid(axis='y', alpha=0.3)

stats.probplot(train[target_col], dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('Q-Q Plot (Normality Test)')

train[target_col].plot(kind='density', ax=axes[1, 1])
axes[1, 1].set_xlabel('Milk Yield (L)')
axes[1, 1].set_title('Milk Yield Distribution - Kernel Density Estimate')
axes[1, 1].axvline(train[target_col].mean(), color='red', linestyle='--', label=f'Mean: {train[target_col].mean():.2f}')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("=" * 60)
print("Target variable statistics")
print("=" * 60)
print(f"Mean: {train[target_col].mean():.4f}")
print(f"Median: {train[target_col].median():.4f}")
print(f"Std: {train[target_col].std():.4f}")
print(f"Skewness: {train[target_col].skew():.4f}")
print(f"Kurtosis: {train[target_col].kurtosis():.4f}")
print(f"Min: {train[target_col].min():.4f}")
print(f"Max: {train[target_col].max():.4f}")

skewness = train[target_col].skew()
if abs(skewness) < 0.5:
    print(f"\nSkewness {skewness:.4f} - Distribution approximately symmetric")
elif abs(skewness) < 1:
    print(f"\nSkewness {skewness:.4f} - Mildly skewed distribution")
else:
    print(f"\nSkewness {skewness:.4f} - Highly skewed distribution; consider log transform")


## Numerical Feature Analysis

Analyze the distributions and statistical properties of all numerical features.

In [ ]:
if len(numeric_features) > 0:
    n_features = len(numeric_features)
    n_cols = 3
    n_rows = min(4, (n_features + n_cols - 1) // n_cols)
    
    for page in range((n_features + 11) // 12):
        start_idx = page * 12
        end_idx = min(start_idx + 12, n_features)
        features_subset = numeric_features[start_idx:end_idx]
        
        n_plots = len(features_subset)
        n_rows_actual = (n_plots + n_cols - 1) // n_cols
        
        fig, axes = plt.subplots(n_rows_actual, n_cols, figsize=(15, 4*n_rows_actual))
        if isinstance(axes, np.ndarray):
            axes = axes.flatten()
        else:
            axes = [axes]
        
        for idx, feature in enumerate(features_subset):
            axes[idx].hist(train[feature].dropna(), bins=30, edgecolor='black', alpha=0.7)
            axes[idx].set_title(f'{feature}\nSkewness: {train[feature].skew():.2f}')
            axes[idx].set_xlabel(feature)
            axes[idx].set_ylabel('Frequency')
        
        for idx in range(n_plots, len(axes)):
            axes[idx].axis('off')
        
        plt.suptitle(f'Numerical Feature Distributions (Page {page+1})', y=1.00, fontsize=14)
        plt.tight_layout()
        plt.show()

In [ ]:
if len(numeric_features) > 0:
    corr_features = numeric_features + [target_col]
    correlation_matrix = train[corr_features].corr()
    
    plt.figure(figsize=(14, 12))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=False, cmap='coolwarm', 
                center=0, square=True, linewidths=0.5, 
                cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Heatmap (Lower Triangle)', fontsize=14, pad=20)
    plt.tight_layout()
    plt.show()
    
    target_correlation = correlation_matrix[target_col].drop(target_col).sort_values(
        key=abs, ascending=False
    ).head(15)
    
    plt.figure(figsize=(10, 8))
    colors = ['green' if x > 0 else 'red' for x in target_correlation.values]
    plt.barh(range(len(target_correlation)), target_correlation.values, color=colors)
    plt.yticks(range(len(target_correlation)), target_correlation.index)
    plt.xlabel('Correlation Coefficient')
    plt.title('Top 15 Features Correlated with Target', fontsize=14)
    plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
    plt.tight_layout()
    plt.show()
    
    print("=" * 60)
    print("Top 15 features most correlated with the target")
    print("=" * 60)
    print(target_correlation)


## Categorical Feature Analysis

Analysis of the distribution of categorical features and their relationship with the target variable.

In [ ]:
if len(categorical_features) > 0:
    print("=" * 60)
    print("Categorical Feature Analysis")
    print("=" * 60)
    
    for feature in categorical_features:
        n_unique = train[feature].nunique()
        print(f"\nFeature: {feature}")
        print(f"  Number of Unique Values: {n_unique}")
        print(f"  Number of Missing Values: {train[feature].isnull().sum()}")
        
        if n_unique <= 10:
            print(f"  Value Distribution:")
            print(train[feature].value_counts().to_string())

In [ ]:
if len(categorical_features) > 0:
    for feature in categorical_features[:min(6, len(categorical_features))]:
        n_unique = train[feature].nunique()
        
        if n_unique <= 20:
            fig, axes = plt.subplots(1, 2, figsize=(15, 5))
            
            value_counts = train[feature].value_counts()
            axes[0].barh(range(len(value_counts)), value_counts.values)
            axes[0].set_yticks(range(len(value_counts)))
            axes[0].set_yticklabels(value_counts.index)
            axes[0].set_xlabel('Count')
            axes[0].set_title(f'{feature} - Category Distribution')
            
            category_means = train.groupby(feature)[target_col].mean().sort_values(ascending=True)
            axes[1].barh(range(len(category_means)), category_means.values)
            axes[1].set_yticks(range(len(category_means)))
            axes[1].set_yticklabels(category_means.index)
            axes[1].set_xlabel('Average Milk Production')
            axes[1].set_title(f'{feature} - Average Milk Production by Category')
            
            plt.tight_layout()
            plt.show()

## Outlier Detection

Using the IQR method to detect outliers:
- IQR = Q3 - Q1
- Lower Bound = Q1 - 1.5 * IQR
- Upper Bound = Q3 + 1.5 * IQR

In [ ]:
if len(numeric_features) > 0:
    outlier_summary = []
    
    for feature in numeric_features:
        Q1 = train[feature].quantile(0.25)
        Q3 = train[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = train[(train[feature] < lower_bound) | (train[feature] > upper_bound)]
        n_outliers = len(outliers)
        outlier_percent = (n_outliers / len(train)) * 100
        
        if n_outliers > 0:
            outlier_summary.append({
                'feature': feature,
                'n_outliers': n_outliers,
                'percent': outlier_percent,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound,
                'min_value': train[feature].min(),
                'max_value': train[feature].max()
            })
    
    if outlier_summary:
        outlier_df = pd.DataFrame(outlier_summary).sort_values('percent', ascending=False)
        print("=" * 80)
        print("Outlier Detection Results (IQR Method)")
        print("=" * 80)
        print(outlier_df.to_string(index=False))
        
        top_outliers = outlier_df.head(6)
        if len(top_outliers) > 0:
            n_plots = len(top_outliers)
            n_cols = 3
            n_rows = (n_plots + n_cols - 1) // n_cols
            
            fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 4*n_rows))
            if n_rows == 1:
                axes = [axes] if n_plots == 1 else axes
            else:
                axes = axes.flatten()
            
            for plot_idx, (df_idx, row) in enumerate(top_outliers.iterrows()):
                feature = row['feature']
                axes[plot_idx].boxplot(train[feature].dropna(), vert=True)
                axes[plot_idx].set_ylabel(feature)
                axes[plot_idx].set_title(f'{feature}\nOutliers: {row["n_outliers"]} ({row["percent"]:.2f}%)')
                axes[plot_idx].grid(axis='y', alpha=0.3)
            
            for idx in range(n_plots, len(axes)):
                axes[idx].axis('off')
            
            plt.suptitle('Features with Most Outliers (Boxplot)', y=1.00, fontsize=14)
            plt.tight_layout()
            plt.show()
    else:
        print("No outliers detected using the IQR method")

## 2. Data Cleaning

In this section, we will:
1. Handle missing values (different strategies)
2. Handle outliers (truncation or removal)
3. Optimize data types to save memory

**Strategies**:
- **Missing values**: 
  - Numerical: Choose mean/median based on skewness
  - Categorical: Use mode
  - Create missing value indicators
- **Outliers**: 
  - Use Winsorization (truncation) method
  - Only handle highly skewed features

In [ ]:
missing_threshold = 5

for col in train.columns:
    missing_pct = (train[col].isnull().sum() / len(train)) * 100
    if missing_pct > missing_threshold:
        indicator_name = f'{col}_missing'
        train[indicator_name] = train[col].isnull().astype(int)
        test[indicator_name] = test[col].isnull().astype(int)
        print(f"Created missing value indicator: {indicator_name} (Missing rate: {missing_pct:.2f}%)")


In [ ]:
for feature in numeric_features:
    if train[feature].isnull().sum() > 0:
        skewness = train[feature].skew()
        
        if abs(skewness) > 1:
            fill_value = train[feature].median()
            strategy = "Median (skewed distribution)"
        else:
            fill_value = train[feature].mean()
            strategy = "Mean (normal distribution)"
        
        train[feature].fillna(fill_value, inplace=True)
        test[feature].fillna(fill_value, inplace=True)
        
        print(f"{feature}: Filled using {strategy}, Value={fill_value:.4f}, Skewness={skewness:.4f}")

In [ ]:
for feature in categorical_features:
    if train[feature].isnull().sum() > 0:
        mode_value = train[feature].mode()[0]
        train[feature].fillna(mode_value, inplace=True)
        test[feature].fillna(mode_value, inplace=True)
        print(f"{feature}: Filled using mode, Value={mode_value}")

print("\n" + "=" * 60)
print("Missing value imputation completed")
print("=" * 60)
print(f"Remaining missing values in training set: {train.isnull().sum().sum()}")
print(f"Remaining missing values in test set: {test.isnull().sum().sum()}")

In [ ]:
outlier_treatment_summary = []

for feature in numeric_features:
    skewness = abs(train[feature].skew())
    
    if skewness > 2:
        Q1 = train[feature].quantile(0.25)
        Q3 = train[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        n_outliers_before = ((train[feature] < lower_bound) | (train[feature] > upper_bound)).sum()
        
        if n_outliers_before > 0:
            train[feature] = train[feature].clip(lower=lower_bound, upper=upper_bound)
            test[feature] = test[feature].clip(lower=lower_bound, upper=upper_bound)
            
            outlier_treatment_summary.append({
                'feature': feature,
                'skewness': skewness,
                'n_outliers': n_outliers_before,
                'lower_bound': lower_bound,
                'upper_bound': upper_bound
            })

if outlier_treatment_summary:
    outlier_df = pd.DataFrame(outlier_treatment_summary)
    print("=" * 80)
    print("Outlier Treatment (Winsorization) - Only for Highly Skewed Features (Skewness > 2)")
    print("=" * 80)
    print(outlier_df.to_string(index=False))
else:
    print("No outlier treatment needed (no highly skewed features)")

In [ ]:
for col in train.select_dtypes(include=['int64']).columns:
    if col != id_col:
        c_min = train[col].min()
        c_max = train[col].max()
        
        if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
            train[col] = train[col].astype(np.int8)
            test[col] = test[col].astype(np.int8)
        elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
            train[col] = train[col].astype(np.int16)
            test[col] = test[col].astype(np.int16)
        elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            train[col] = train[col].astype(np.int32)
            test[col] = test[col].astype(np.int32)

for col in train.select_dtypes(include=['float64']).columns:
    if col != target_col:
        c_min = train[col].min()
        c_max = train[col].max()
        
        if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            train[col] = train[col].astype(np.float32)
            test[col] = test[col].astype(np.float32)

print("=" * 60)
print("Data type optimization completed")
print("=" * 60)

## 3. Feature Engineering

Feature engineering is key to improving model performance! We will implement:

1. **Frequency Encoding** - Replace categorical values with their occurrence frequency
2. **Target Encoding** - Encode using the mean of the target variable
3. **Label Encoding** - Convert categories to numbers (for tree models)
4. **Interaction Features** - Create interaction terms between features
5. **Polynomial Features** - Create squares, cubes, etc. of features
6. **Statistical Features** - Grouped statistics by category

In [ ]:
print("=" * 60)
print("1. Frequency Encoding")
print("=" * 60)

for feature in categorical_features:
    freq_encoding = train[feature].value_counts(normalize=True).to_dict()
    
    new_feature_name = f'{feature}_freq'
    train[new_feature_name] = train[feature].map(freq_encoding)
    test[new_feature_name] = test[feature].map(freq_encoding)
    
    min_freq = train[new_feature_name].min()
    test[new_feature_name].fillna(min_freq, inplace=True)
    
    print(f"{feature} -> {new_feature_name}")

In [ ]:
print("\n" + "=" * 60)
print("2. Target Encoding with Smoothing")
print("=" * 60)

smoothing = 10
global_mean = train[target_col].mean()

for feature in categorical_features:
    agg = train.groupby(feature)[target_col].agg(['mean', 'count'])
    
    smoothed_mean = (agg['mean'] * agg['count'] + global_mean * smoothing) / (agg['count'] + smoothing)
    target_encoding = smoothed_mean.to_dict()
    
    new_feature_name = f'{feature}_target'
    train[new_feature_name] = train[feature].map(target_encoding)
    test[new_feature_name] = test[feature].map(target_encoding)
    
    test[new_feature_name].fillna(global_mean, inplace=True)
    
    print(f"{feature} -> {new_feature_name} (smoothing={smoothing})")

In [ ]:
print("\n" + "=" * 60)
print("3. Label Encoding")
print("=" * 60)

label_encoders = {}

for feature in categorical_features:
    le = LabelEncoder()
    
    combined = pd.concat([train[feature], test[feature]], axis=0)
    le.fit(combined)
    
    new_feature_name = f'{feature}_label'
    train[new_feature_name] = le.transform(train[feature])
    test[new_feature_name] = le.transform(test[feature])
    
    label_encoders[feature] = le
    print(f"{feature} -> {new_feature_name} ({len(le.classes_)} categories)")

In [ ]:
print("\n" + "=" * 60)
print("4. Feature Interactions")
print("=" * 60)

numeric_features_updated = train.select_dtypes(include=['int8', 'int16', 'int32', 'int64', 
                                                         'float32', 'float64']).columns.tolist()
if target_col in numeric_features_updated:
    numeric_features_updated.remove(target_col)
if id_col in numeric_features_updated:
    numeric_features_updated.remove(id_col)

if len(numeric_features_updated) > 0:
    correlations = train[numeric_features_updated + [target_col]].corr()[target_col].drop(target_col)
    top_features = correlations.abs().sort_values(ascending=False).head(5).index.tolist()
    
    print(f"Selected top 5 features: {top_features}")
    
    interaction_count = 0
    for i in range(len(top_features)):
        for j in range(i+1, len(top_features)):
            feat1, feat2 = top_features[i], top_features[j]
            
            train[f'{feat1}_x_{feat2}'] = train[feat1] * train[feat2]
            test[f'{feat1}_x_{feat2}'] = test[feat1] * test[feat2]
            
            train[f'{feat1}_div_{feat2}'] = train[feat1] / (train[feat2] + 1e-5)
            test[f'{feat1}_div_{feat2}'] = test[feat1] / (test[feat2] + 1e-5)
            
            train[f'{feat1}_plus_{feat2}'] = train[feat1] + train[feat2]
            test[f'{feat1}_plus_{feat2}'] = test[feat1] + test[feat2]
            
            train[f'{feat1}_minus_{feat2}'] = train[feat1] - train[feat2]
            test[f'{feat1}_minus_{feat2}'] = test[feat1] - test[feat2]
            
            interaction_count += 4
    
    print(f"Created {interaction_count} interaction features")


In [ ]:
print("\n" + "=" * 60)
print("5. Polynomial Features")
print("=" * 60)

if len(numeric_features_updated) > 0:
    correlations = train[numeric_features_updated + [target_col]].corr()[target_col].drop(target_col)
    top_10_features = correlations.abs().sort_values(ascending=False).head(10).index.tolist()
    
    print(f"Selected top 10 features: {top_10_features}")
    
    for feature in top_10_features:
        train[f'{feature}_squared'] = train[feature] ** 2
        test[f'{feature}_squared'] = test[feature] ** 2
        
        train[f'{feature}_cubed'] = train[feature] ** 3
        test[f'{feature}_cubed'] = test[feature] ** 3
        
        if train[feature].min() >= 0:
            train[f'{feature}_sqrt'] = np.sqrt(train[feature])
            test[f'{feature}_sqrt'] = np.sqrt(test[feature])
    
    print(f"Created polynomial features (squared, cubed, square root) for top 10 features")


In [ ]:
print("\n" + "=" * 60)
print("6. Statistical Aggregations")
print("=" * 60)

if len(categorical_features) > 0 and len(numeric_features_updated) > 0:
    top_numeric = correlations.abs().sort_values(ascending=False).head(5).index.tolist()
    
    for cat_feature in categorical_features[:min(3, len(categorical_features))]:
        for num_feature in top_numeric:
            grouped_mean = train.groupby(cat_feature)[num_feature].mean()
            train[f'{cat_feature}_{num_feature}_mean'] = train[cat_feature].map(grouped_mean)
            test[f'{cat_feature}_{num_feature}_mean'] = test[cat_feature].map(grouped_mean)
            test[f'{cat_feature}_{num_feature}_mean'].fillna(train[num_feature].mean(), inplace=True)
            
            grouped_std = train.groupby(cat_feature)[num_feature].std()
            train[f'{cat_feature}_{num_feature}_std'] = train[cat_feature].map(grouped_std)
            test[f'{cat_feature}_{num_feature}_std'] = test[cat_feature].map(grouped_std)
            test[f'{cat_feature}_{num_feature}_std'].fillna(train[num_feature].std(), inplace=True)
            
            grouped_median = train.groupby(cat_feature)[num_feature].median()
            train[f'{cat_feature}_{num_feature}_median'] = train[cat_feature].map(grouped_median)
            test[f'{cat_feature}_{num_feature}_median'] = test[cat_feature].map(grouped_median)
            test[f'{cat_feature}_{num_feature}_median'].fillna(train[num_feature].median(), inplace=True)
    
    print(f"Created statistical aggregation features (mean, std, median)")


In [ ]:
train.to_csv(PROCESSED_DATA_DIR / 'train_processed.csv', index=False)
test.to_csv(PROCESSED_DATA_DIR / 'test_processed.csv', index=False)

print("\n" + "=" * 60)
print("Data saved successfully")
print("=" * 60)
print(f"Training set shape: {train.shape}")
print(f"Test set shape: {test.shape}")
print(f"Number of features increased: {train.shape[1] - len(numeric_features) - len(categorical_features) - 2}")
print(f"\nSave paths:")
print(f"  - {PROCESSED_DATA_DIR / 'train_processed.csv'}")
print(f"  - {PROCESSED_DATA_DIR / 'test_processed.csv'}")


## 4. Feature Selection & Dimensionality Reduction

Before training the model, we need to:
1. Remove original categorical features (already encoded)
2. Prepare training and validation sets
3. Remove highly correlated features (to avoid multicollinearity)
4. Feature scaling (standardization)
5. PCA dimensionality reduction

In [ ]:
categorical_features = train.select_dtypes(include=['object']).columns.tolist()
if id_col in categorical_features:
    categorical_features.remove(id_col)

print("=" * 60)
print("Remove original categorical features (already encoded)")
print("=" * 60)
print(f"Features to remove: {categorical_features}")

train.drop(columns=categorical_features, inplace=True, errors='ignore')
test.drop(columns=categorical_features, inplace=True, errors='ignore')

print(f"Removal completed")
print(f"Training set shape: {train.shape}")
print(f"Test set shape: {test.shape}")

In [ ]:
X = train.drop(columns=[target_col, id_col], errors='ignore')
y = train[target_col]
X_test = test.drop(columns=[id_col], errors='ignore')
test_ids = test[id_col]

print("=" * 60)
print("Features and target variable preparation completed")
print("=" * 60)
print(f"Feature matrix X shape: {X.shape}")
print(f"Target variable y shape: {y.shape}")
print(f"Test set X_test shape: {X_test.shape}")
print(f"Number of features: {X.shape[1]}")

missing_cols = set(X.columns) - set(X_test.columns)
extra_cols = set(X_test.columns) - set(X.columns)

if missing_cols:
    print(f"Test set is missing columns: {missing_cols}")
    for col in missing_cols:
        X_test[col] = 0

if extra_cols:
    print(f"Test set has extra columns: {extra_cols}")
    X_test = X_test[X.columns]

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("=" * 60)
print("Dataset split completed")
print("=" * 60)
print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")
print(f"\nTraining set proportion: {len(X_train)/len(X)*100:.1f}%")
print(f"Validation set proportion: {len(X_val)/len(X)*100:.1f}%")

In [ ]:
correlation_matrix = X_train.corr().abs()

upper_triangle = correlation_matrix.where(
    np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool)
)

high_corr_threshold = 0.95
to_drop = [column for column in upper_triangle.columns 
           if any(upper_triangle[column] > high_corr_threshold)]

print("=" * 60)
print(f"Removing highly correlated features (correlation > {high_corr_threshold})")
print("=" * 60)
print(f"Found {len(to_drop)} highly correlated features")

if to_drop:
    print(f"Features to drop: {to_drop[:10]}...")
    
    X_train = X_train.drop(columns=to_drop)
    X_val = X_val.drop(columns=to_drop)
    X_test = X_test.drop(columns=to_drop)
    
    print(f"Removal completed")
    print(f"New training set shape: {X_train.shape}")
else:
    print("No highly correlated features found")

In [ ]:
print("\n" + "=" * 60)
print("StandardScaler")
print("=" * 60)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_val_scaled = pd.DataFrame(X_val_scaled, columns=X_val.columns, index=X_val.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("StandardScaler completed")
print(f"Training set mean: {X_train_scaled.mean().mean():.6f}")
print(f"Training set std deviation: {X_train_scaled.std().mean():.6f}")

joblib.dump(scaler, MODEL_DIR / 'scaler.pkl')
print(f"Scaler saved to: {MODEL_DIR / 'scaler.pkl'}")

## Principal Component Analysis (PCA)

Principal Component Analysis (PCA) can:
1. Reduce feature dimensionality
2. Remove noise
3. Visualize feature importance
4. Accelerate model training

We will analyze how many principal components are needed to retain 95% of the variance.

In [ ]:
pca_full = PCA(random_state=RANDOM_STATE)
pca_full.fit(X_train_scaled)

cumsum_variance = np.cumsum(pca_full.explained_variance_ratio_)

n_components_95 = np.argmax(cumsum_variance >= 0.95) + 1

print("=" * 60)
print("PCA Analysis Results")
print("=" * 60)
print(f"Original number of features: {X_train_scaled.shape[1]}")
print(f"Number of principal components to retain 95% variance: {n_components_95}")
print(f"Dimensionality reduction ratio: {(1 - n_components_95/X_train_scaled.shape[1])*100:.1f}%")
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(range(1, min(51, len(pca_full.explained_variance_ratio_)+1)), 
             pca_full.explained_variance_ratio_[:50], 'bo-')
axes[0].set_xlabel('Principal Component')
axes[0].set_ylabel('Explained Variance Ratio')
axes[0].set_title('Scree Plot - First 50 Principal Components')
axes[0].grid(alpha=0.3)

axes[1].plot(range(1, min(101, len(cumsum_variance)+1)), 
             cumsum_variance[:100], 'ro-')
axes[1].axhline(y=0.95, color='green', linestyle='--', label='95% Variance')
axes[1].axvline(x=n_components_95, color='blue', linestyle='--', 
                label=f'{n_components_95} Principal Components')
axes[1].set_xlabel('Number of Principal Components')
axes[1].set_ylabel('Cumulative Explained Variance Ratio')
axes[1].set_title('Cumulative Explained Variance Plot')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
pca = PCA(n_components=n_components_95, random_state=RANDOM_STATE)

X_train_pca = pca.fit_transform(X_train_scaled)
X_val_pca = pca.transform(X_val_scaled)
X_test_pca = pca.transform(X_test_scaled)

print("=" * 60)
print("PCA Dimensionality Reduction Completed")
print("=" * 60)
print(f"Original number of features: {X_train_scaled.shape[1]}")
print(f"Number of features after reduction: {X_train_pca.shape[1]}")
print(f"Variance retained: {pca.explained_variance_ratio_.sum()*100:.2f}%")
joblib.dump(pca, MODEL_DIR / 'pca.pkl')
print(f"PCA model saved")

## 5. Model Training

We will train the following 7 regression models:

1. **Linear Regression** - The simplest baseline model
2. **Ridge Regression** - L2 regularization to prevent overfitting
3. **Lasso Regression** - L1 regularization for feature selection
4. **Decision Tree** - Non-linear model
5. **K-Nearest Neighbors (KNN)** - Distance-based method
6. **Support Vector Regression (SVR)** - Support vector regression
7. **Neural Network (MLP)** - Multi-layer perceptron

Each model will undergo hyperparameter tuning and cross-validation.

In [ ]:
def evaluate_model(model, X_train, y_train, X_val, y_val, model_name):
    y_train_pred = model.predict(X_train)
    train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
    train_mae = mean_absolute_error(y_train, y_train_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    
    y_val_pred = model.predict(X_val)
    val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
    val_mae = mean_absolute_error(y_val, y_val_pred)
    val_r2 = r2_score(y_val, y_val_pred)
    
    overfitting = train_rmse < val_rmse * 0.9
    
    print("=" * 60)
    print(f"{model_name} - Evaluation Results")
    print("=" * 60)
    print(f"Training RMSE: {train_rmse:.4f}")
    print(f"Validation RMSE: {val_rmse:.4f}")
    print(f"Training MAE:  {train_mae:.4f}")
    print(f"Validation MAE:  {val_mae:.4f}")
    print(f"Training R²:   {train_r2:.4f}")
    print(f"Validation R²:   {val_r2:.4f}")
    
    if overfitting:
        print(f"Warning: Possible overfitting detected!")
    else:
        print(f"Model generalizes well")
    
    return {
        'model_name': model_name,
        'train_rmse': train_rmse,
        'val_rmse': val_rmse,
        'train_mae': train_mae,
        'val_mae': val_mae,
        'train_r2': train_r2,
        'val_r2': val_r2,
        'overfitting': overfitting
    }


In [ ]:
all_models_results = []
trained_models = {}

### 1. Linear Regression

The most basic regression model, using the least squares method.
- **Advantages**: Simple, highly interpretable
- **Disadvantages**: Assumes a linear relationship, may underfit

In [ ]:
print("\n" + "Training Linear Regression model...\n")

lr_model = LinearRegression()
lr_model.fit(X_train_scaled, y_train)

lr_results = evaluate_model(lr_model, X_train_scaled, y_train, 
                             X_val_scaled, y_val, "Linear Regression")

all_models_results.append(lr_results)
trained_models['linear_regression'] = lr_model

feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': np.abs(lr_model.coef_)
}).sort_values('coefficient', ascending=False).head(10)

print(f"\nTop 10 most important features (by absolute coefficient value):")
print(feature_importance.to_string(index=False))

### 2. Ridge Regression (L2 Regularization)

Ridge regression adds an L2 regularization term to prevent overfitting.
- **Regularization term**: λ * Σ(w²)
- **Hyperparameter to tune**: alpha (λ)

In [ ]:
print("\n" + "Training Ridge Regression model (alpha selected by cross-validation)...\n")

alphas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
cv_scores = []

for alpha in alphas:
    ridge = Ridge(alpha=alpha, random_state=RANDOM_STATE)
    scores = cross_val_score(ridge, X_train_scaled, y_train, 
                            cv=5, scoring='neg_root_mean_squared_error')
    cv_scores.append(-scores.mean())

best_alpha = alphas[np.argmin(cv_scores)]

print(f"Alpha test results:")
for alpha, score in zip(alphas, cv_scores):
    marker = " ← Best" if alpha == best_alpha else ""
    print(f"  alpha={alpha:7.3f}, CV RMSE={score:.4f}{marker}")

ridge_model = Ridge(alpha=best_alpha, random_state=RANDOM_STATE)
ridge_model.fit(X_train_scaled, y_train)

ridge_results = evaluate_model(ridge_model, X_train_scaled, y_train, 
                               X_val_scaled, y_val, "Ridge Regression")

all_models_results.append(ridge_results)
trained_models['ridge'] = ridge_model

print(f"\nBest alpha: {best_alpha}")

### 3. Lasso Regression (L1 Regularization)

Lasso regression uses L1 regularization and can perform feature selection.
- **Regularization term**: λ * Σ|w|
- **Characteristic**: Can shrink some coefficients to zero, achieving feature selection

In [ ]:
print("\n" + "Training Lasso Regression model (alpha selected by cross-validation)...\n")

alphas_lasso = [0.001, 0.01, 0.1, 1, 10]
cv_scores_lasso = []

for alpha in alphas_lasso:
    lasso = Lasso(alpha=alpha, random_state=RANDOM_STATE, max_iter=10000)
    scores = cross_val_score(lasso, X_train_scaled, y_train, 
                            cv=5, scoring='neg_root_mean_squared_error')
    cv_scores_lasso.append(-scores.mean())

best_alpha_lasso = alphas_lasso[np.argmin(cv_scores_lasso)]

print(f"Alpha test results:")
for alpha, score in zip(alphas_lasso, cv_scores_lasso):
    marker = " ← Best" if alpha == best_alpha_lasso else ""
    print(f"  alpha={alpha:7.3f}, CV RMSE={score:.4f}{marker}")

lasso_model = Lasso(alpha=best_alpha_lasso, random_state=RANDOM_STATE, max_iter=10000)
lasso_model.fit(X_train_scaled, y_train)

lasso_results = evaluate_model(lasso_model, X_train_scaled, y_train, 
                               X_val_scaled, y_val, "Lasso Regression")

all_models_results.append(lasso_results)
trained_models['lasso'] = lasso_model

n_nonzero = np.sum(lasso_model.coef_ != 0)
print(f"\nFeature selection results:")
print(f"  Original number of features: {len(lasso_model.coef_)}")
print(f"  Number of non-zero coefficients: {n_nonzero}")
print(f"  Number of features removed: {len(lasso_model.coef_) - n_nonzero}")

### 4. Decision Tree Regression

Decision trees make predictions by recursively partitioning the feature space.
- **Splitting criterion**: MSE (Mean Squared Error)
- **Important parameter**: max_depth (maximum depth of the tree)
- **Advantages**: Can capture nonlinear relationships

In [ ]:
print("\n" + "Training Decision Tree model (selecting depth by cross-validation)...\n")

max_depths = [3, 5, 7, 10, 15, 20, None]
cv_scores_dt = []

for depth in max_depths:
    dt = DecisionTreeRegressor(max_depth=depth, random_state=RANDOM_STATE)
    scores = cross_val_score(dt, X_train_scaled, y_train, 
                            cv=5, scoring='neg_root_mean_squared_error')
    cv_scores_dt.append(-scores.mean())

best_depth = max_depths[np.argmin(cv_scores_dt)]

print(f"Max depth test results:")
for depth, score in zip(max_depths, cv_scores_dt):
    marker = " ← Best" if depth == best_depth else ""
    depth_str = "None (unlimited)" if depth is None else str(depth)
    print(f"  max_depth={depth_str:15s}, CV RMSE={score:.4f}{marker}")

dt_model = DecisionTreeRegressor(max_depth=best_depth, random_state=RANDOM_STATE)
dt_model.fit(X_train_scaled, y_train)

dt_results = evaluate_model(dt_model, X_train_scaled, y_train, 
                            X_val_scaled, y_val, "Decision Tree")

all_models_results.append(dt_results)
trained_models['decision_tree'] = dt_model

feature_importance_dt = pd.DataFrame({
    'feature': X_train.columns,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False).head(10)

print(f"\nTop 10 Important Features (Decision Tree):")
print(feature_importance_dt.to_string(index=False))

### 5. K-Nearest Neighbors (KNN) Regression

KNN predicts the target value by finding the k nearest neighbors.
- **Distance Metric**: Euclidean distance
- **Key Parameter**: k (number of neighbors)
- **Prediction**: Average value of the k nearest neighbors

In [ ]:
print("\n" + "Training KNN model (selecting k by cross-validation)...\n")

k_values = [3, 5, 7, 10, 15, 20, 25, 30]
cv_scores_knn = []

for k in k_values:
    knn = KNeighborsRegressor(n_neighbors=k)
    scores = cross_val_score(knn, X_train_scaled, y_train, 
                            cv=5, scoring='neg_root_mean_squared_error')
    cv_scores_knn.append(-scores.mean())

best_k = k_values[np.argmin(cv_scores_knn)]

print(f"k value test results:")
for k, score in zip(k_values, cv_scores_knn):
    marker = " ← Best" if k == best_k else ""
    print(f"  k={k:3d}, CV RMSE={score:.4f}{marker}")

knn_model = KNeighborsRegressor(n_neighbors=best_k)
knn_model.fit(X_train_scaled, y_train)

knn_results = evaluate_model(knn_model, X_train_scaled, y_train, 
                             X_val_scaled, y_val, "KNN")

all_models_results.append(knn_results)
trained_models['knn'] = knn_model

print(f"\nBest k value: {best_k}")

### 6. Support Vector Regression (SVR)

SVR uses kernel tricks to map data into a high-dimensional space.
- **Kernel Function**: RBF (Radial Basis Function)
- **Characteristics**: Suitable for small to medium-sized datasets
- **Note**: May be slow for large datasets

In [ ]:
print("\n" + "Training SVR model (RBF kernel)...\n")

sample_size = min(10000, len(X_train_scaled))
sample_indices = np.random.choice(len(X_train_scaled), sample_size, replace=False)

X_train_sample = X_train_scaled.iloc[sample_indices]
y_train_sample = y_train.iloc[sample_indices]

print(f"Using {sample_size} samples to train SVR (accelerated training)")

svr_model = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr_model.fit(X_train_sample, y_train_sample)

svr_results = evaluate_model(svr_model, X_train_scaled, y_train, 
                             X_val_scaled, y_val, "SVR")

all_models_results.append(svr_results)
trained_models['svr'] = svr_model


### 7. Neural Network Regression (MLP)

Neural networks can learn complex nonlinear patterns.
- **Architecture**: Input layer → Hidden layers → Output layer
- **Activation Function**: ReLU
- **Optimizer**: Adam
- **Regularization**: L2 regularization + Early Stopping

In [ ]:
print("\n" + "MLP\n")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Use device: {device}")
if torch.cuda.is_available():
    print(f"GPU model: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB\n")

class MLPRegressorPyTorch(nn.Module):
    def __init__(self, input_size, hidden_layers):
        super(MLPRegressorPyTorch, self).__init__()
        layers = []
        prev_size = input_size
        
        for hidden_size in hidden_layers:
            layers.append(nn.Linear(prev_size, hidden_size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(0.2))
            prev_size = hidden_size
        
        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

def train_pytorch_mlp(model, train_loader, val_loader, epochs=200, lr=0.001, patience=10, use_validation=True):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=0.001)
    
    best_train_loss = float('inf')
    patience_counter = 0
    epochs_trained = 0
    
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X_batch.size(0)
        
        train_loss /= len(train_loader.dataset)
        epochs_trained = epoch + 1
        
        if use_validation and val_loader is not train_loader:
            model.eval()
            val_loss = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    outputs = model(X_batch)
                    loss = criterion(outputs, y_batch)
                    val_loss += loss.item() * X_batch.size(0)
            val_loss /= len(val_loader.dataset)
            monitor_loss = val_loss
        else:
            monitor_loss = train_loss
        
        if monitor_loss < best_train_loss:
            best_train_loss = monitor_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            model.load_state_dict(best_model_state)
            break
    
    return model, best_train_loss, epochs_trained

architectures = [(100,), (100, 50), (100, 100), (100, 50, 25)]
input_size = X_train_scaled.shape[1]

X_train_tensor = torch.FloatTensor(X_train_scaled.values)
y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_val_tensor = torch.FloatTensor(X_val_scaled.values)
y_val_tensor = torch.FloatTensor(y_val.values).reshape(-1, 1)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False)

best_score = float('inf')
best_architecture = None
best_model_state = None

print("Testing different network architectures:")
for arch in architectures:
    model = MLPRegressorPyTorch(input_size, arch).to(device)
    model, val_loss, n_epochs = train_pytorch_mlp(model, train_loader, val_loader, epochs=200, lr=0.001, patience=10)
    
    score = np.sqrt(val_loss)
    
    if score < best_score:
        best_score = score
        best_architecture = arch
        best_model_state = model.state_dict().copy()
    
    marker = " ← Best" if score == best_score else ""
    print(f"  Architecture={str(arch):20s}, Validation RMSE={score:.4f}, Epochs={n_epochs}{marker}")

print(f"\nUsing the best architecture: {best_architecture}")

mlp_model = MLPRegressorPyTorch(input_size, best_architecture).to(device)
mlp_model.load_state_dict(best_model_state)
final_loss = best_score ** 2
final_epochs = 0

class PyTorchModelWrapper:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.loss_ = final_loss
        self.n_iter_ = 500
    
    def predict(self, X):
        self.model.eval()
        with torch.no_grad():
            X_tensor = torch.FloatTensor(X.values if hasattr(X, 'values') else X).to(self.device)
            predictions = self.model(X_tensor).cpu().numpy().flatten()
        return predictions

mlp_model_wrapper = PyTorchModelWrapper(mlp_model, device)

mlp_results = evaluate_model(mlp_model_wrapper, X_train_scaled, y_train, 
                             X_val_scaled, y_val, "MLP")

all_models_results.append(mlp_results)
trained_models['mlp'] = mlp_model_wrapper

print(f"\nTraining info:")
print(f"  Final loss: {mlp_model_wrapper.loss_:.4f}")
print(f"  Best architecture: {best_architecture}")
print(f"  Number of hidden layers: {len(best_architecture)}")
print(f"  Device used: {device}")

## 6. Ensemble Learning

Ensemble learning improves performance by combining multiple models. We will implement:

1. **Random Forest** - Bagging method
2. **Gradient Boosting** - Boosting method
3. **AdaBoost** - Adaptive boosting
4. **Voting Regressor** - Voting ensemble
5. **Stacking Regressor** - Stacking ensemble

In [ ]:
print("\n" + "Random Forest\n")

from sklearn.model_selection import RandomizedSearchCV

rf_param_dist = {
    'n_estimators': [150, 200, 250],
    'max_depth': [12, 15, 18], 
    'min_samples_split': [20, 50, 100],
    'min_samples_leaf': [10, 20, 30],
    'max_features': [0.5, 0.7, 'sqrt'],
    'max_samples': [0.7, 0.8, 0.9]
}

import os
n_cores = max(1, os.cpu_count() - 2) if os.cpu_count() else 4

rf = RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=n_cores)

rf_grid_search = RandomizedSearchCV(
    rf, rf_param_dist, 
    n_iter=10,
    cv=3, 
    scoring='neg_root_mean_squared_error',
    n_jobs=1,
    verbose=2,
    random_state=RANDOM_STATE
)

rf_grid_search.fit(X_train_scaled, y_train)

print(f"\nBest Parameters:")
for param, value in rf_grid_search.best_params_.items():
    print(f"  {param}: {value}")

rf_model = rf_grid_search.best_estimator_

rf_results = evaluate_model(rf_model, X_train_scaled, y_train, 
                            X_val_scaled, y_val, "Random Forest")

all_models_results.append(rf_results)
trained_models['random_forest'] = rf_model

feature_importance_rf = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False).head(15)

print(f"\nTop 15 Important Features (Random Forest):")
print(feature_importance_rf.to_string(index=False))

In [ ]:
print("Training Gradient Boosting Regressor (faster version)...\n")
gb_model = GradientBoostingRegressor(
    n_estimators=150,
    learning_rate=0.05,
    max_depth=5,
    min_samples_split=30,
    min_samples_leaf=15,
    subsample=0.8,
    max_features='sqrt',
    validation_fraction=0.1,
    n_iter_no_change=20,
    random_state=RANDOM_STATE,
    verbose=1
)

gb_model.fit(X_train_scaled, y_train)

gb_results = evaluate_model(gb_model, X_train_scaled, y_train, 
                            X_val_scaled, y_val, "Gradient Boosting")

all_models_results.append(gb_results)
trained_models['gradient_boosting'] = gb_model

In [ ]:
print("Training AdaBoost Regressor (optimized version)...\n")

ada_model = AdaBoostRegressor(
    estimator=DecisionTreeRegressor(
        max_depth=4,
        min_samples_split=30,
        min_samples_leaf=15,
        max_features='sqrt',
        random_state=RANDOM_STATE
    ),
    n_estimators=100,
    learning_rate=0.3,
    loss='square',
    random_state=RANDOM_STATE
)

ada_model.fit(X_train_scaled, y_train)

ada_results = evaluate_model(ada_model, X_train_scaled, y_train, 
                             X_val_scaled, y_val, "AdaBoost")

all_models_results.append(ada_results)
trained_models['adaboost'] = ada_model

In [ ]:
print("Training Voting Regressor (ensemble of Ridge, GB, RF, MLP)...\n")
voting_model = VotingRegressor(
    estimators=[
        ('ridge', ridge_model),
        ('gb', gb_model),
        ('rf', rf_model),
        ('mlp', mlp_model_wrapper)
    ],
    weights=[1, 2, 2, 1]
)

voting_model.fit(X_train_scaled, y_train)

voting_results = evaluate_model(voting_model, X_train_scaled, y_train, 
                                X_val_scaled, y_val, "Voting Regressor")

all_models_results.append(voting_results)

trained_models['voting'] = voting_model

In [ ]:
print("Training Stacking Regressor (faster version)...\n")
base_learners = [
    ('rf', RandomForestRegressor(
        n_estimators=50,
        max_depth=8,
        max_features='sqrt',
        min_samples_split=20,
        n_jobs=-1,
        random_state=RANDOM_STATE
    )),
    ('gb', GradientBoostingRegressor(
        n_estimators=50,
        max_depth=3,
        learning_rate=0.2,
        subsample=0.8,
        max_features='sqrt',
        random_state=RANDOM_STATE
    )),
    ('ridge', Ridge(alpha=best_alpha if 'best_alpha' in globals() else 1.0, random_state=RANDOM_STATE))
]

meta_learner = Ridge(alpha=1.0, random_state=RANDOM_STATE)

stacking_model = StackingRegressor(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv=3,
    n_jobs=1
)

stacking_model.fit(X_train_scaled, y_train)

stacking_results = evaluate_model(stacking_model, X_train_scaled, y_train, 
                                  X_val_scaled, y_val, "Stacking Regressor")

all_models_results.append(stacking_results)
trained_models['stacking'] = stacking_model
print("Stacking Regressor training completed!")

## 7. Model Evaluation & Comparison

In [ ]:
results_df = pd.DataFrame(all_models_results)
results_df = results_df.sort_values('val_rmse')

print("=" * 80)
print("Model Performance Comparison (sorted by validation RMSE)")
print("=" * 80)
print(results_df.to_string(index=False))

best_model_name = results_df.iloc[0]['model_name']
best_model_val_rmse = results_df.iloc[0]['val_rmse']

print("\n" + "=" * 80)
print(f"Best Model: {best_model_name}")
print(f"Validation RMSE: {best_model_val_rmse:.4f}")
print("=" * 80)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 10))

axes[0].barh(results_df['model_name'], results_df['val_rmse'])
axes[0].set_xlabel('Validation RMSE')
axes[0].set_title('Model Performance Comparison - Validation RMSE (Lower is Better)')
axes[0].invert_yaxis()
axes[0].grid(axis='x', alpha=0.3)

axes[1].barh(results_df['model_name'], results_df['val_r2'], color='green')
axes[1].set_xlabel('Validation R²')
axes[1].set_title('Model Performance Comparison - Validation R² (Higher is Better)')
axes[1].invert_yaxis()
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

fig, ax = plt.subplots(figsize=(10, 8))

ax.scatter(results_df['train_rmse'], results_df['val_rmse'], s=100, alpha=0.6)

for idx, row in results_df.iterrows():
    ax.annotate(row['model_name'], 
                (row['train_rmse'], row['val_rmse']),
                xytext=(5, 5), textcoords='offset points',
                fontsize=8)

min_val = min(results_df['train_rmse'].min(), results_df['val_rmse'].min())
max_val = max(results_df['train_rmse'].max(), results_df['val_rmse'].max())
ax.plot([min_val, max_val], [min_val, max_val], 'r--', alpha=0.5, label='Ideal Line')

ax.set_xlabel('Training RMSE')
ax.set_ylabel('Validation RMSE')
ax.set_title('Training vs Validation Performance (Detecting Overfitting)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 8. Final Prediction and Submission

Using the best model:
1. Retrain on the full training set
2. Predict on the test set
3. Generate a Kaggle submission file

In [ ]:
print("\n" + "=" * 80)
print("Retraining Best Model on Full Training Set")
print("=" * 80)

X_full = pd.concat([X_train_scaled, X_val_scaled])
y_full = pd.concat([y_train, y_val])

print(f"Full training set size: {X_full.shape}")

best_model_key = None
for key, results in zip(trained_models.keys(), all_models_results):
    if results['model_name'] == best_model_name:
        best_model_key = key
        break

if best_model_key == 'stacking':
    final_model = StackingRegressor(
        estimators=base_learners,
        final_estimator=meta_learner,
        cv=5
    )
elif best_model_key == 'random_forest':
    final_model = rf_model
elif best_model_key == 'gradient_boosting':
    final_model = gb_model
else:
    final_model = trained_models[best_model_key]

print(f"Retraining model: {best_model_name}")
final_model.fit(X_full, y_full)
print("Training completed")

model_filename = MODEL_DIR / f'final_model_{best_model_key}.pkl'
joblib.dump(final_model, model_filename)
print(f"Model saved to: {model_filename}")

In [ ]:
print("\n" + "=" * 80)
print("Generating Predictions on Test Set")
print("=" * 80)

test_predictions = final_model.predict(X_test_scaled)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
submission = pd.DataFrame({
    id_col: test_ids,
    target_col: test_predictions
})

submission_filename = SUBMISSION_DIR / f'submission_{best_model_key}_{timestamp}.csv'
submission.to_csv(submission_filename, index=False)

print(f"Submission file generated!")
print(f"File path: {submission_filename}")
print(f"\nPrediction Statistics:")
print(f"  Mean: {test_predictions.mean():.4f}")
print(f"  Median: {np.median(test_predictions):.4f}")
print(f"  Standard Deviation: {test_predictions.std():.4f}")
print(f"  Minimum: {test_predictions.min():.4f}")
print(f"  Maximum: {test_predictions.max():.4f}")

print(f"\nFirst 10 rows of the submission file:")
print(submission.head(10).to_string(index=False))

In [ ]:
results_df.to_csv(MODEL_DIR / 'model_comparison.csv', index=False)

summary = {
    'project_name': 'Kaggle Regression Project',
    'completion_date': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'best_model': best_model_name,
    'best_val_rmse': float(best_model_val_rmse),
    'n_features_original': len(numeric_features) + len(categorical_features),
    'n_features_engineered': X_full.shape[1],
    'n_models_trained': len(all_models_results),
    'submission_file': str(submission_filename)
}

import json
with open(MODEL_DIR / 'project_summary.json', 'w', encoding='utf-8') as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)
print(json.dumps(summary, indent=2, ensure_ascii=False))
